Simplified Molecular Input Line Entry System (SMILES) representation

In [ ]:
!conda install -c rdkit rdkit -y

In [ ]:
import os
os.listdir("../input/smiles-toxicity/data/data/NR-ER-train")

In [ ]:
import pandas as pd
filename = "../input/smiles-toxicity/data/data/NR-ER-train/names_smiles.csv"
names = ['id', 'sequence']

data_df = pd.read_csv(filename, names = names)

raw_text = '\n'.join(data_df['sequence'].tolist())

print(raw_text)

In [ ]:
import pandas as pd
filenamey = "../input/smiles-toxicity/data/data/NR-ER-train/names_labels.csv"
names = ['id', 'label']

data_dfy = pd.read_csv(filenamey, names = names)['label']




In [ ]:
data_dfy.values

In [ ]:
from rdkit import Chem
m = Chem.MolFromSmiles('CCC1=NC=CN=C1C')
m


In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import matplotlib.pyplot as plt

# Chem.MolFromSmiles('c1ccccccccccc1')
#m = Chem.MolFromSmiles('c1ccccccccccc1')
#Draw.MolToMPL(d, size=(200, 200))
#m = Chem.MolFromSmiles('C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=C3O2)Cl)Cl')
x = 'O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5'
m = Chem.MolFromSmiles(x)
Draw.MolToMPL(m, size=(100, 100))

#m

In [ ]:
unique_chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(unique_chars))
#char_to_int.update({-1 : "\n"})
char_to_int

In [ ]:
n_chars = len(raw_text)
n_vocab = len(unique_chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
   seq_in = raw_text[i:i + seq_length]
   seq_out = raw_text[i + seq_length]
   dataX.append([char_to_int[char] for char in seq_in])
   dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns) 

In [ ]:
import numpy as np
from keras.utils import np_utils
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X=X/ float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
y

In [ ]:
len(X)

In [ ]:
len(y) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation= 'softmax' ))
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam', metrics = ['accuracy'])
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor= 'loss' , verbose=1, save_best_only=True,
mode= 'min' )
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=5, batch_size=1000, callbacks=callbacks_list)

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(unique_chars))

In [ ]:
import numpy as np
import sys
from keras.models import load_model
# load the network weights
filename = "weights-improvement-05-1.7667.hdf5"
model.load_weights(filename)
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' )
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"") 
# generate characters
for i in range(1000):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x=x/ float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.") 